In [ ]:
####################################################################
# bertwordpiece 방식으로 토큰 사전 생성 
#
# 1) mecab 로 한번 말뭉치 분석
# 2) 분석된 말뭉치를 가지고 wordpiece 토큰화 
# 3) special token 추가
####################################################################

#변수들 지정 
#
# input : 말뭉치 경로
# mecab_output : 입력된 말뭉치를 mecab 으로 형태소 분석한 output 파일 
# mecab_generation = True 이면, 형태소 앞에 ## 붙임, False면 안붙임

input = '../../korpora/mycorpus/bong_corpus.txt'
mecab_output = '../../tokenizer/my_bong_vocab/bong_corpus_mecab.txt' # ["wiki_20190620_small.txt", "my_data/nsmc_subword.txt"], 여러개 파일도 추가할수 있음
mecab_generation = False

vocab_folder = '../tokenizer'
vocab_output = 'wiki_20190620_false_0311'
vocab_special_output = 'wiki_20190620_false_0311_speical'

vocab_size = 32000
min_frequency = 5

In [ ]:
#import utils
#from utils import seed_everything

#seed 설정 해도 vocab 은 다르게 생성됨
#seed_everything(111)

In [ ]:
##### Mecab 를 이용하여, 말뭉치 형태소 분석 
import konlpy
from konlpy.tag import Mecab
from tqdm.notebook import tqdm

In [ ]:
with open(input, 'r', encoding='utf-8') as f:
    data = f.read().split('\n')

In [ ]:
print(data[:3])
print(len(data))

In [ ]:
mecab = Mecab()

In [ ]:
# True: '어릴때' -> '어릴', '##'때' 로 일반화 시킴
#sentence_count = 0
if mecab_generation == True:
  total_morph=[]
  for sentence in tqdm(data):
    morph_sentence=[]
    count=0
    for token_mecab in mecab.morphs(sentence):
      token_mecab_save = token_mecab
      if count > 0:
        token_mecab_save = "##" + token_mecab_save
        morph_sentence.append(token_mecab_save)
      else:
        morph_sentence.append(token_mecab_save)
        count += 1
    total_morph.append(morph_sentence)  
    
    #if sentence_count % 1000000 == 0:
    #    print('sentence_count:{}'.format(sentence_count))
    
    #sentence_count += 1
        
# False: '어릴때' -> '어릴', '때' 로 일반화 시킴
else:
  total_morph=[]
  for sentence in tqdm(data):
    morph_sentence = mecab.morphs(sentence)
    total_morph.append(morph_sentence)
   
    #if sentence_count % 1000000 == 0:
    #   print('sentence_count:{}'.format(sentence_count))
    
    #sentence_count += 1

In [ ]:
print(total_morph[:3])
print(len(total_morph))

In [ ]:
#sentence_count = 0
with open(mecab_output, 'w', encoding='utf-8') as f:
  for line in tqdm(total_morph):
    # ** 앞에 ' ' 공백을 넣어서 형태로 별로 분리하여 저장함
    f.write(' '.join(line)+'\n')

In [ ]:
##### Bert Tokenizer 
from tokenizers import BertWordPieceTokenizer

In [ ]:
WPT = BertWordPieceTokenizer(
    #vocab='/content/wordPieceTokenizer/wiki_20190620-vocab.txt', # 기존 vocab 넣어도 사전 학습 안됨
    clean_text=True,       # 토큰에서 공백은 제거 
    handle_chinese_chars=True,  # 한자는 모두 char 단위로 쪼갬
    strip_accents=False,  # True로 하면, 가자 => ㄱ ㅏ ㅈ ㅏ 식으로 토큰화 되어 버림(*따라서 한국어에서는 반드시 False)
    lowercase=False       # 한국어에서는 반드시 False
)

In [ ]:
WPT.train(
    #files=["my_data/wiki_20190620_small.txt", "my_data/nsmc_subword.txt"],  #여러개 파일도 추가할수 있음
    files=[mecab_output],
    vocab_size=vocab_size,       # 만들고자 하는 vocab의 size, 보통 '32000' 정도가 좋다고 알려져 있다
    min_frequency=min_frequency,        # merge를 수행할 최소 빈도수, 2로 설정 시 2회 이상 등장한 pair만 수행한다
    show_progress=True,     # 학습 진행과정 show
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],# Tokenizer에 추가하고 싶은 special token 지정
    wordpieces_prefix="##"
    #limit_alphabet=6000  # merge 수행 전 initial tokens이 유지되는 숫자 제한
    #initial_alphabet   # 꼭 포함됐으면 하는 initial alphabet, 이곳에 설정한 token은 학습되지 않고 그대로 포함되도록 설정된다.
)

In [ ]:
# vocab 파일 저장
WPT.save_model(vocab_folder, vocab_output)

In [ ]:
print(WPT.get_vocab_size())

In [ ]:
text = "오늘은 경마장에서 조랑말 탄다"
tokenized_text = WPT.encode(text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

In [ ]:
##### speical 토큰 추가 및 json 파일 생성
from transformers import BertTokenizer
from transformers import BertTokenizerFast

In [ ]:
vocab_path = vocab_folder + '/' + vocab_output + '-vocab.txt'
vocab_special_path =  vocab_folder + '/' + vocab_special_output

In [ ]:
print(vocab_path)
print(vocab_special_path)

In [ ]:
# BertTokenizerFast, BertTokenizer 둘중 하나를 사용하면 됨
'''
tokenizer = BertTokenizerFast(vocab_file=vocab_path, 
                              max_len=128, 
                              do_lower_case=False)
'''                              

#print(tokenizer.tokenize("뷁은 [MASK] 조선 중기의 무신이다."))
#tokenizer.add_special_tokens({'mask_token':'[MASK]'})

tokenizer = BertTokenizer(vocab_file=vocab_path, 
                          strip_accents=False, 
                          do_lower_case=False)

In [ ]:
"""Special Token 지정

-special_tokens : 언어모델의 범용성을 위해 Dummy token, 여러 개의 [unused]와 [UNK]를 꼭 설정해야 한다.

-언어모델에 번역, 요약, 개체명 인식 모델을 fine-tuning시 Dummy token이 필요한 경우가 많다. 또 도메인 특화된 task를 수행할 땐 도메인 토큰을 따로 선언하는게 필수이다.

-이를 고려하지 않으면 편법을 사용해야 한다. ETRI의 KorBERT는 Dummy token이 없어 빈도수가 작은 token을 Dummy로 대체해서 쓰기도 한다.

-꼭 충분한 unused와 UNK를 설정하자. 그리고 BOS(문장 시작), EOS(문장 끝) 등등도 추가하자. 본 프로젝트에선 10개의 UNK와 200개의 unsed token을 선언하였다.



"""

special_tokens=['[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]',
                '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]',]


In [ ]:
special_tokens_dict = {'additional_special_tokens': special_tokens}
tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
# special token 체크
tokenizer.all_special_tokens

In [ ]:
# special token 추가한 special vocab을 저장함
tokenizer.save_pretrained(vocab_special_path)

In [ ]:
# special vocab 확인

tokenizer_check = BertTokenizerFast.from_pretrained(vocab_special_path)

print('check special tokens : %s'%tokenizer_check.all_special_tokens[:20])

print('vocab size : %d' % tokenizer_check.vocab_size)
tokenized_input_for_pytorch = tokenizer_check("나는 오늘 아침밥을 먹었다.", return_tensors="pt")
#tokenized_input_for_tensorflow = tokenizer_check("나는 오늘 아침밥을 먹었다.", return_tensors="tf")

print("Tokens (str)      : {}".format([tokenizer_check.convert_ids_to_tokens(s) for s in tokenized_input_for_pytorch['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(tokenized_input_for_pytorch['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(tokenized_input_for_pytorch['attention_mask'].tolist()[0]))
